In [1]:
from bs4 import BeautifulSoup as bs
import requests

url = "https://www.espn.com/tennis/rankings/_/type/wta"
data = requests.get(url).text
soup = bs(data, "html.parser")
ths = soup.find_all("th")
trs = soup.find_all("tr")

player_info_list = []

def clean_tag(soup):
    for tag in soup.find_all("sup"):
        tag.decompose()

def get_player_info(url):
    
    data = requests.get(full_url).text
    soup = bs(data, "html.parser")

    table = soup.find(class_="infobox vcard")
    rows = soup.find_all("tr")

    clean_tag(soup)
    player = {}

    for index, row in enumerate(rows):
        if len(row) == 1:
            continue
        elif index <= 20:
            try:
                dict_key = row.find("th").get_text(" ", strip=True).replace("\xa0", " ").replace("\n", " ")
                dict_value = row.find("td").get_text(" ", strip=True).replace("\xa0", " ").replace("\n", " ")
                player[dict_key] = dict_value
            except Exception as e:
                
                pass
        else:
            continue

    return player


for index, tr in enumerate(trs):
    player_info = {}
    if index == 0:
        continue
    else:
        tds = tr.find_all("td")
        row = [td.get_text() for td in tds]
        
        player_info["Rank"] = row[0]
        player_name = row[2]
        player_info["Name"] = player_name
        player_info["Points"] = row[3]
                
        wiki_url = "https://en.wikipedia.org/wiki/"
        relative_url = player_name.replace(" ", "_")
        full_url = wiki_url + relative_url
        
        player = get_player_info(full_url)
        for k, v in player.items():
            player_info[k] = v
    
    player_info_list.append(player_info)



In [2]:
import re

birth_date = r"\d{4}-\d{2}-\d{2}"
percentage = r"\d\d\.\d"
number = r"\d+(,\d{3})*\.*\d*"
height= r"\d\.\d{2}"
height2=r"\d{3}"
url=r"(http)?(://)?(www.)?[a-zA-Z0-9 -._]*\.\w+"

In [3]:
def get_birthplace(phrase):
    if phrase is None:
        return None
    else:
        return " ".join(str(phrase).split(" ")[8:])
    
def get_birthday(phrase):
    if re.search(birth_date, str(phrase)) is not None:
        return re.search(birth_date, str(phrase)).group()
    else:
        return None

def get_height(phrase):
    if re.search(height, str(phrase)) is not None:
        return float(re.search(height, phrase).group())
    elif re.search(height2, str(phrase)) is not None:
        return int(re.search(height2, phrase).group())/100
    else:
        return None
    
def get_percentage(phrase):
    if re.search(percentage, str(phrase)) is not None:
        return float(re.search(percentage, phrase).group())
    else:
        return None
    
def get_prize(money):
    if re.search(number, str(money)) is not None:
        return int(re.search(number, money).group().replace(",", ""))
    else:
        return None  
    
def get_website(phrase):
    if re.search(url, str(phrase)) is not None:
        return re.search(url, str(phrase)).group()
    else:
        return None

In [4]:
import pandas as pd
df = pd.DataFrame(player_info_list)

df["Birthday"]=df["Born"].apply(lambda x: get_birthday(x))
df["Birthplace"]=df["Born"].apply(lambda x: get_birthplace(x))
df["height m"]=df["Height"].apply(lambda x: get_height(x))
df["Win Percentage"]=df["Career record"].apply(lambda x: get_percentage(x))
df["prize"]=df["Prize money"].apply(lambda x: get_prize(x))
df["website"]=df["Official website"].apply(lambda x: get_website(x))

In [5]:
from datetime import datetime as dt
today = dt.now().strftime("%y_%m_%d")
df.to_excel(f"WTA_Player_{today}.xlsx", index=False, sheet_name=f"Top150_WTA_{today}")

In [6]:
df.head


<bound method NDFrame.head of     Rank                Name Points  \
0      1         Iga Swiatek  8,631   
1      2     Anett Kontaveit  4,511   
2      3        Paula Badosa  4,245   
3      4          Ons Jabeur  4,150   
4      5     Aryna Sabalenka  4,145   
..   ...                 ...    ...   
145  146          Qiang Wang    441   
146  147  Anastasia Gasanova    432   
147  148            Yue Yuan    431   
148  149     Arina Rodionova    428   
149  150         Fiona Ferro    427   

                                Country (sports)              Residence  \
0                                         Poland        Raszyn , Poland   
1                                        Estonia       Viimsi , Estonia   
2                                          Spain            Dubai , UAE   
3                                        Tunisia                  Tunis   
4                                        Belarus  Miami , Florida, U.S.   
..                                           ...   